In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [3]:
!pip uninstall -y wandb

Found existing installation: wandb 0.19.5
Uninstalling wandb-0.19.5:
  Successfully uninstalled wandb-0.19.5


In [4]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b5e8e4d1473f4dbaf74cc569028e5fd847ec725eb2838719c86c55aac2ff7a4e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [5]:
!pip install rank_bm25

In [17]:
# main_notebook.ipynb
import importlib
import os
import sys
import pandas as pd
import numpy as np
import torch
from ast import literal_eval

# Add the NLU and converters directories to the system path for imports
sys.path.append(os.path.abspath("NLU"))
sys.path.append(os.path.abspath("converters"))
sys.path.append(os.path.abspath("PolicyManager"))
sys.path.append(os.path.abspath("NLG"))

import intent_detection
import slot_filling
import policy_manager
import Gemini_NLG
import HuggingFace_NLG
import IR_module
import Translator
import Entity_Detector

importlib.reload(intent_detection)
importlib.reload(slot_filling)
importlib.reload(policy_manager)
importlib.reload(Gemini_NLG)
importlib.reload(HuggingFace_NLG)
importlib.reload(Translator)
importlib.reload(Entity_Detector)
importlib.reload(IR_module)

from intent_detection import train_intent_detection, predict_intent
from slot_filling import train_slot_filling, predict_slots
from Entity_Detector import train_entity_detector, predict_entities
from policy_manager import PolicyManager
from Gemini_NLG import GeminiNLG
from HuggingFace_NLG import NLG_API
from IR_module import IRModule
from Translator import Translator


# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

In [9]:
import json

with open('label_slot_dict.json', 'r') as file:
    label_slot_dict = json.load(file)

In [10]:
def train_chatbot(
    intent_train_df,
    intent_val_df,
    slot_train_df,
    slot_val_df,
    entity_train_df,
    entity_val_df,
    nlu_model_name="HooshvareLab/bert-base-parsbert-uncased",
    output_dir="./nlu_model",
    num_epochs=3,
    batch_size=32,
    learning_rate=1e-5,
    early_stopping_patience=2,
    max_length=160
):

    print("Starting chatbot training process...\n")

    # 1. Train Intent Detection (NLU) Module
    intent_detection_sub_dir = output_dir + "/intent_detection_module"
    train_intent_detection(
        train_df=intent_train_df,
        val_df=intent_val_df,
        model_name=nlu_model_name,
        output_dir=intent_detection_sub_dir,
        num_epochs=num_epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        early_stopping_patience=early_stopping_patience,
        max_length=max_length
    )

    print("\nIntent Detection (NLU) Module trained successfully!\n")

    # 2. Train Slot filling module
    slot_filling_sub_dir = output_dir + "/slot_filling_module"
    train_slot_filling(
        train_df=slot_train_df,
        val_df=slot_val_df,
        label_slot_dict=label_slot_dict,
        model_name=nlu_model_name,
        output_dir=slot_filling_sub_dir,
        num_train_epochs=num_epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        max_length=max_length
    )
    print("\Slot filling Module trained successfully!\n")

    entity_detector_sub_dir = output_dir + "/entity_detector"
    train_entity_detector(
        entity_train_df,
        entity_val_df,
        model_name=nlu_model_name,
        output_dir=entity_detector_sub_dir,
        num_train_epochs=num_epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        max_length=max_length,
    )
    print("\Entity Detector module trained successfully.\n")

    print("All modules trained successfully!")

In [11]:
intent_train_df = pd.read_csv('train.csv')
intent_val_df = pd.read_csv('validation.csv')
slot_train_df = pd.read_csv('train.csv')
slot_val_df = pd.read_csv('validation.csv')
entity_train_df = pd.read_csv('entity_detection_train.csv')
entity_val_df = pd.read_csv('entity_detection_val.csv')
for df in [entity_train_df, entity_val_df]:
    df['tokens'] = df['tokens'].apply(literal_eval)
    df['labels'] = df['labels'].apply(literal_eval)
train_chatbot(intent_train_df=intent_train_df, intent_val_df=intent_val_df, slot_train_df=slot_train_df, slot_val_df=slot_val_df, entity_train_df=entity_train_df, entity_val_df=entity_val_df, num_epochs=3)

Starting chatbot training process...

Encoding labels...
Checking dataset balance...
Label 'balance_bill' (ID: 0) has 90 samples.
Label 'card2card' (ID: 1) has 88 samples.
Label 'change_password' (ID: 2) has 120 samples.
Label 'close_card' (ID: 3) has 120 samples.
Label 'convert_cheque' (ID: 4) has 88 samples.
Label 'currency_request' (ID: 5) has 91 samples.
Label 'delegate_account' (ID: 6) has 90 samples.
Label 'duplicate_card' (ID: 7) has 119 samples.
Label 'installment_payment' (ID: 8) has 90 samples.
Label 'loan_free' (ID: 9) has 78 samples.
Label 'loan_interest' (ID: 10) has 72 samples.
Label 'open_account_current' (ID: 11) has 79 samples.
Label 'open_account_deposit' (ID: 12) has 74 samples.
Label 'open_account_free' (ID: 13) has 92 samples.
Label 'paya' (ID: 14) has 92 samples.
Label 'receipt_payment' (ID: 15) has 79 samples.
Label 'recieve_cheque' (ID: 16) has 120 samples.
Label 'signin_problem' (ID: 17) has 71 samples.
Label 'software_problem' (ID: 18) has 110 samples.
Label '

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1943 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.051460,0.983564,0.953060,0.968072,0.982407
2,No log,0.028499,0.991446,0.971500,0.981372,0.989679
3,No log,0.026450,0.991475,0.974853,0.983094,0.990617


\Entity Detector module trained successfully.

All modules trained successfully!


In [18]:
def finalize_inference(nlg_module, input_dict):
    # Translate to persian for user ...
    try:
        persian_dict = nlg_module.translator.translate_dict_to_persian(input_dict)
    except Exception as e:
        print(f"Error translating the dictionary: {e}")
        return input_dict

    loop_counter = 0
    current_dict = persian_dict

    while loop_counter < 5: # limit number of iterations ...
        # Ask the user to finalize it...
        print("\nمطمئن هستید که اطلاعات زیر صحیح است؟")
        for key, value in current_dict['parameters'].items():
            print(f"{key}: {value}")

        print("\nآیا این اطلاعات صحیح است؟ (بلی یا نه)")
        user_response = input()

        if user_response.strip() == "بلی":
            print("اطلاعات تایید شد.")
            current_dict = nlg_module.translator.translate_dict_to_english(current_dict)
            return current_dict
        elif user_response.strip() == "نه":
            loop_counter += 1
            print("لطفا بگویید چه چیزی باید تغییر کند و چرا:")
            user_input = input()

            # Translate it back to eng and use nlg to update the field ...
            english_dict = nlg_module.translator.translate_dict_to_english(current_dict)
            updated_dict = nlg_module.update_dict_based_on_input(english_dict, user_input)
            if updated_dict is None:
                print("خطا در بروزرسانی دیکشنری. تغییرات ذخیره نشد.")
                return input_dict

            # Translate updated dict to persian and consider it as new dict
            updated_persian_dict = nlg_module.translator.translate_dict_to_persian(updated_dict)
            updated_persian_dict['intent'] = current_dict['intent']
            current_dict['parameters'] = updated_persian_dict['parameters']
        else:
            print("پاسخ نامعتبر. لطفاً 'بلی' یا 'نه' وارد کنید.")
            loop_counter += 1

    final_dict = nlg_module.translator.translate_dict_to_english(current_dict)
    print("اطلاعات نهایی:")
    print(final_dict)
    return final_dict

def confirm_intent(nlg_module, intent_detected, fa_intent):
    while True:
        # Ask for confirmation
        print(f'آیا قصد شما {fa_intent} است؟ (بلی/خیر)')
        confirmation = input().strip()

        if confirmation == "بلی":
            return intent_detected, fa_intent

        elif confirmation == "خیر":
            print('با عرض پوزش از اشتباه پیش آمده، لطفا درخواست خود را به صورت کامل بیان کنید:')
            new_input = input().strip()
            intent_detected, fa_intent = nlg_module.detect_intent(new_input)

        else:
            print('لطفاً فقط یکی از گزینه‌های زیر را وارد کنید:')
            print('- "بلی" برای تایید نیتمندی تشخیص داده شده')
            print('- "خیر" برای وارد کردن درخواست جدید')


def inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict, nlu_dir="./nlu_model"):
    intent_list = list(label_slot_dict.keys())

    init_text = ' سلام. چطور میتوانم کمک تان کنم؟'
    print(init_text)
    user_input = input()
    question = ""
    prev_turn_conversation = user_input
    next_slot_to_be_filled = None

    # Load modules ....
    policy_manager = PolicyManager(intent_list, label_slot_dict, "Others")
    ir_module = IRModule(train_df, 'text', 'label', 'slots')

    # NLG MODULE OPTIONS (GEMINI OR HUGGING_FACE)
    nlg_module = GeminiNLG(GEMINI_API_KEY)
    # nlg_module = NLG_API(HUGGING_FACE_API_KEY)

    # intent detection ...
    intent_detected = predict_intent(user_input, model_dir=nlu_dir + "/intent_detection_module")
    fa_intent,_ = nlg_module.translator.translate(intent_detected, None)

    # check the intent ...
    intent_detected, fa_intent = confirm_intent(nlg_module, intent_detected, fa_intent)

    # set intent and slots ...
    policy_manager.set_intent(intent_detected)
    possible_slots = policy_manager.get_slots_for_intent()

    while True:
        slot_asked_counts = 0
        if next_slot_to_be_filled is not None:
            slot_asked_counts = policy_manager.get_slot_ask_counts()[next_slot_to_be_filled]

        if slot_asked_counts < 2:
            slot_tags = predict_slots(prev_turn_conversation, model_dir=nlu_dir + "/slot_filling_module", intent=intent_detected, possible_slots=possible_slots)
            policy_manager.update_slots(slot_tags)
        elif slot_asked_counts < 3:
            print(f'entity detector is used!') # if slot filling is not finding the answer, use entity detector ..
            entity_tags = predict_entities(prev_turn_conversation, max_length=200)
            policy_manager.update_slots_by_entity_detector(entity_tags, next_slot_to_be_filled)
        else:
            print(f'get it as slot')
            policy_manager.update_slot_based_on_input(user_input, next_slot_to_be_filled)


        # Get remaining slots
        possible_slots = policy_manager.get_remaining_slots()

        # Check if all slots are filled
        if not possible_slots:
            print("All required information has been collected.")
            dialogue_state_dict = policy_manager.get_dialogue_state()
            # print(dialogue_state_dict)
            try:
                finalized_dict = finalize_inference(nlg_module, dialogue_state_dict)
            except Exception as e:
                print(f'Error occured in finalized_inference function! {str(e)}')
                finalized_dict = dialogue_state_dict
            return finalized_dict

        # next question slot
        print(f'next_slot_to_be_filled:{next_slot_to_be_filled}')
        previous_slot = next_slot_to_be_filled
        print(f'previous slot: {previous_slot}')
        next_slot_to_be_filled = policy_manager.get_next_action()
        possible_slots = [next_slot_to_be_filled]
        print(f'possible slots: {possible_slots}')

        # generation part ...
        guided_mode = False
        if previous_slot == next_slot_to_be_filled:
            guided_mode = True

        print(f'guided mode: {str(guided_mode)}')
        examples = ir_module.get_examples(user_input, policy_manager.current_intent, next_slot_to_be_filled)
        if guided_mode == False:
            response, fa_slot = nlg_module.generate_slot_response(question, user_input, intent_detected, next_slot_to_be_filled, ir_examples=examples)
        else:
            guided_mode_examples = ir_module.get_guided_examples(user_input, policy_manager.current_intent, next_slot_to_be_filled)
            response, fa_slot = nlg_module.generate_slot_response(question, user_input, intent_detected, next_slot_to_be_filled, ir_examples=examples,  guided_mode=True, guided_mode_examples=examples)

        # conversation ...
        print(f'response: {response}')
        user_input = input()
        question = f" مقدار {fa_slot} چیست؟ "
        prev_turn_conversation = question + user_input

    return None

In [ ]:
# حساب قرض الحسنه
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام میخواستم حساب قرض الحسنه باز کنم
Input Text: سلام میخواستم حساب قرض الحسنه باز کنم
Predicted Intent: open_account_free
intent:open_account_free, slot:None
translated: افتتاح حساب آزاد, 
آیا قصد شما افتتاح حساب آزاد است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('میخواستم', 'o'), ('حساب', 'o'), ('قرض', 'o'), ('الحسنه', 'o'), ('باز', 'o'), ('کنم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['issuance_card']
guided mode: False
intent:open_account_free, slot:issuance_card
translated: افتتاح حساب آزاد, صدور کارت
response: از پاسخ شما متشکرم. 

آیا تمایل به دریافت کارت بانکی همراه با افتتاح حساب دارید؟  در صورت تمایل، لطفاً پاسخ خود را به صورت مختصر و مفید، مثلاً با عبارت "بله" یا "خیر" ارائه دهید.

بله کارت شتاب صادر شود
slot filling tag results: [('مقدار', 'o'), ('صدور', 'o'), ('کارت', 'o'), ('چیست؟', 'o'), ('بله', 'o'), ('کارت', 'o'), ('شتاب', 'i-issuance_card'), ('صادر', 'o'), ('شود', 'o')]
next_slot_to_be_fi

In [ ]:
# کارت به کارت
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام. میخواستم کارت به کارت کنم
Input Text: سلام. میخواستم کارت به کارت کنم
Predicted Intent: duplicate_card
intent:duplicate_card, slot:None
translated: صدور مجدد کارت, 
آیا قصد شما صدور مجدد کارت است؟ (بلی/خیر)
خیر
با عرض پوزش از اشتباه پیش آمده، لطفا درخواست خود را به صورت کامل بیان کنید:
کارت به کارت میخوام انجام بدم
آیا قصد شما انتقال وجه کارت به کارت است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام.', 'o'), ('میخواستم', 'o'), ('کارت', 'o'), ('به', 'o'), ('کارت', 'o'), ('کنم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['transfer_amount']
guided mode: False
intent:card2card, slot:transfer_amount
translated: انتقال کارت به کارت, مبلغ انتقال
response: از پاسخ شما متشکرم. 

لطفا مبلغ مورد نظر برای انتقال کارت به کارت را به صورت عددی (مثلا: 1500000)  بنویسید.

20000000
slot filling tag results: [('مقدار', 'o'), ('مبلغ', 'o'), ('انتقال', 'o'), ('چیست؟', 'o'), ('20000000', 'o')]
next_slot_to_be_filled:transfer_amount
previo

In [ ]:
# پرداخت قبض
train_dبf = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام میخواستم قبض هام رو پرداخت کنم
Input Text: سلام میخواستم قبض هام رو پرداخت کنم
Predicted Intent: receipt_payment
intent:receipt_payment, slot:None
translated: پرداخت قبض, 
آیا قصد شما پرداخت قبض است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('میخواستم', 'o'), ('قبض', 'o'), ('هام', 'o'), ('رو', 'o'), ('پرداخت', 'o'), ('کنم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['bill_id']
guided mode: False
intent:receipt_payment, slot:bill_id
translated: پرداخت قبض, شناسه قبض
response: از پاسخ شما متشکرم. 

لطفاً شناسه قبض خود را برای پرداخت اعلام فرمایید.  برای مثال، می‌توانید آن را به صورت "شناسه قبض: 12345"  یا "شناسه‌ی قبض من 67890 است"  وارد کنید.

شناسه قبض 12345678 است
slot filling tag results: [('مقدار', 'o'), ('شناسه', 'o'), ('قبض', 'o'), ('چیست؟', 'o'), ('شناسه', 'o'), ('قبض', 'o'), ('12345678', 'b-bill_id'), ('است', 'o')]
next_slot_to_be_filled:bill_id
previous slot: bill_id
possible slots: ['payment_id']


In [ ]:
# صورت حساب مانده
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام وقتت بخیر. میخواستم بپرسم صورتت حساب مانده ام چقدره
Input Text: سلام وقتت بخیر. میخواستم بپرسم صورتت حساب مانده ام چقدره
Predicted Intent: signin_problem
intent:signin_problem, slot:None
translated: مشکل ورود به سیستم, 
آیا قصد شما مشکل ورود به سیستم است؟ (بلی/خیر)
خیر
با عرض پوزش از اشتباه پیش آمده، لطفا درخواست خود را به صورت کامل بیان کنید:
صورت حساب مانده ام را میخواستم بپرسم
آیا قصد شما صورت مانده است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('وقتت', 'o'), ('بخیر.', 'o'), ('میخواستم', 'o'), ('بپرسم', 'o'), ('صورتت', 'o'), ('حساب', 'o'), ('مانده', 'o'), ('ام', 'o'), ('چقدره', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['balance_datetime']
guided mode: False
intent:balance_bill, slot:balance_datetime
translated: صورت حساب مانده, زمان بررسی مانده
response: از پاسخ شما متشکرم. 

برای بررسی مانده حساب، به چه تاریخی نیاز دارید؟

راهنما:  تاریخ مورد نظر خود را به صورت سال-ماه-روز (مثلاً: 1402-05-10)  یا سال-

In [ ]:
# password_change
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام میخواستم پسوردم رو عوض کنم 
Input Text: سلام میخواستم پسوردم رو عوض کنم 
Predicted Intent: signin_problem
intent:signin_problem, slot:None
translated: مشکل ورود به سیستم, 
آیا قصد شما مشکل ورود به سیستم است؟ (بلی/خیر)
خیر
با عرض پوزش از اشتباه پیش آمده، لطفا درخواست خود را به صورت کامل بیان کنید:
تعویض پسورد کارت 
آیا قصد شما تغییر رمز عبور کارت است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('میخواستم', 'o'), ('پسوردم', 'o'), ('رو', 'o'), ('عوض', 'o'), ('کنم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['card_number']
guided mode: False
intent:change_password, slot:card_number
translated: تغییر رمز, شماره کارت
response: از پاسخ شما متشکرم. 

لطفاً شماره کارت خود را به فرمت ۱۶ رقمی  وارد کنید.  به عنوان مثال،  `۶۲۲۱۰۰۰۰۰۰۰۰۰۰۰`.

6219111122223333
slot filling tag results: [('مقدار', 'o'), ('شماره', 'o'), ('کارت', 'o'), ('چیست؟', 'o'), ('6219111122223333', 'b-card_number')]
next_slot_to_be_filled:card_number


In [ ]:
# دریافت چک
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام وقتت بخیر. میخواستم چک بگیرم
Input Text: سلام وقتت بخیر. میخواستم چک بگیرم
Predicted Intent: recieve_cheque
intent:recieve_cheque, slot:None
translated: دریافت چک, 
آیا قصد شما دریافت چک است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('وقتت', 'o'), ('بخیر.', 'o'), ('میخواستم', 'o'), ('چک', 'o'), ('بگیرم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['sayad_id']
guided mode: False
intent:recieve_cheque, slot:sayad_id
translated: دریافت چک, شناسه صیاد
response: از پاسخ شما متشکرم. 

لطفاً شناسه صیاد چک دریافتی خود را به من بگویید.

راهنما:  شناسه صیاد یک عدد است که روی چک درج شده است.  کافی است  این عدد را به صورت عددی  به من اطلاع دهید.

ممنونم. شناسه صیاد چکم 2315513 هستش
slot filling tag results: [('مقدار', 'o'), ('شناسه', 'o'), ('صیاد', 'o'), ('چیست؟', 'o'), ('ممنونم.', 'o'), ('شناسه', 'o'), ('صیاد', 'o'), ('چکم', 'o'), ('2315513', 'b-sayad_id'), ('هستش', 'o')]
next_slot_to_be_filled:sayad_id
previous slot:

In [20]:
# پرداخت اقساط
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
سلام میخواستم پرداخت قسطم رو انجام بدم.
Input Text: سلام میخواستم پرداخت قسطم رو انجام بدم.
Predicted Intent: installment_payment
intent:installment_payment, slot:None
translated: پرداخت اقساط, 
آیا قصد شما پرداخت اقساط است؟ (بلی/خیر)
بلی
slot filling tag results: [('سلام', 'o'), ('میخواستم', 'o'), ('پرداخت', 'o'), ('قسطم', 'o'), ('رو', 'o'), ('انجام', 'o'), ('بدم.', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['installment_amount']
guided mode: False
intent:installment_payment, slot:installment_amount
translated: پرداخت اقساط, مبلغ قسط
response: از پاسخ شما متشکرم. 

لطفاً مبلغ قسطی که قصد پرداخت آن را دارید، به صورت عددی و به ریال یا تومان اعلام فرمایید.  مثلاً: "5000000 ریال" یا "5000000 تومان".

50000000 ریال
slot filling tag results: [('مقدار', 'o'), ('مبلغ', 'o'), ('قسط', 'o'), ('چیست؟', 'o'), ('50000000', 'b-installment_amount'), ('ریال', 'i-installment_amount')]
next_slot_to_be_filled:installment_amount
previous slot: i

In [21]:
#  مسدود سازی کارت
train_df = pd.read_csv('train.csv')
inference_output = inference_chatbot_gemini(train_df, GEMINI_API_KEY, HUGGING_FACE_API_KEY, label_slot_dict)
print(f'inference output: {inference_output}')

 سلام. چطور میتوانم کمک تان کنم؟
میخوام کارتم رو مسدود کنم
Input Text: میخوام کارتم رو مسدود کنم
Predicted Intent: close_card
intent:close_card, slot:None
translated: مسدودسازی کارت, 
آیا قصد شما مسدودسازی کارت است؟ (بلی/خیر)
بلی
slot filling tag results: [('میخوام', 'o'), ('کارتم', 'o'), ('رو', 'o'), ('مسدود', 'o'), ('کنم', 'o')]
next_slot_to_be_filled:None
previous slot: None
possible slots: ['card_number']
guided mode: False
intent:close_card, slot:card_number
translated: مسدودسازی کارت, شماره کارت
response: از پاسخ شما متشکرم. 

برای مسدود کردن کارتتان، لطفا شماره 16 رقمی کارت خود را به من بگویید.  به عنوان مثال، می‌توانید شماره کارت را به صورت  `XXXXXXXXXXXXXXX`  وارد کنید.

6219123412341234
slot filling tag results: [('مقدار', 'o'), ('شماره', 'o'), ('کارت', 'o'), ('چیست؟', 'o'), ('6219123412341234', 'b-card_number')]
next_slot_to_be_filled:card_number
previous slot: card_number
possible slots: ['blocking_reason']
guided mode: False
intent:close_card, slot:blocking_reason
translat